# Part 2. Notebook for Network and Topological Analysis in Neuroscience

Authors: Eduarda Centeno & Fernando Santos
    
This second part of the project will focus on the 3D visualisations we have developed.


## Table of contents
1. [Imports](#imports)
2. [Short recap - importing data & creating graph object](#short-recap)
3. [Creating 3D visualisations with Plotly](#3d-visualisation)
4. [Acknowledgements](#acknowledgements)

<a id='imports'></a>
###  **1.** Imports

In [2]:
# Basic data manipulation and visualisation libraries
import numpy as np
import matplotlib.pyplot as plt

# Network Libraries
import networkx as nx

# Magic command to load watermark
#%load_ext watermark

In [ ]:
# Print watermark
%watermark --author "Eduarda & Fernando" --date --time --python --machine --iversion --watermark --packages jupyterlab,notebook

<a id='short-recap'></a>
### **2.** Short recap - importing data & creating graph object

In [ ]:
# Import matrix
matrix = np.genfromtxt('AveragedMatrix.txt')

# Absolutise for further user
matrix = abs(matrix)

# Creating Graph
G = nx.from_numpy_matrix(matrix)

# Removing self-loops
G.remove_edges_from(list(nx.selfloop_edges(G)))

# This function accepts a argument 'distance' that, in correlation-based networks, must be seen as the inverse ...
# ... of the weight value. Thus, a high correlation value (e.g. 0.8) means a shorter distance (i.e 0.2).
G_distance_dict = {(e1, e2): 1 / abs(weight) for e1, e2, weight in G.edges(data='weight')}

# Then add them as attributes to the graph
nx.set_edge_attributes(G, G_distance_dict, 'distance')

# Create graphs from comparison
matrix2 = matrix.copy()
matrix3 = matrix.copy()

#####  **Important!!**
Have a look at the following files to check if they match your matrix's regions (they are called internally in the HCP_Data_Vis.py). If not, you will have to create your own files:

    a. *_region_names_abbrev.txt
    b. HCP_positions.txt
    
If you are working with matrices that had regions removed (low signal, tumor mask, etc.), you will have to adapt the files above to take that into account. We aim to soon implement a flexible way of removing regions for this notebook, but it's not done yet. Subnetwork 3D plots will also be available soon.

!<a id='3d-visualisation'></a>
### **3.** Creating 3D visualisations with Plotly
First, we need to import the necessary backup script.
Then, it is possible to use a list of values for most of the nodal metrics, e.g., centralities, etc.

In [7]:
!pip install meshio
%run "HCP_Data_Vis.py"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.2/166.2 kB 3.2 MB/s eta 0:00:00


/content/HCP_Data_Vis.py:1376: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

/content/HCP_Data_Vis.py:1377: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

/content/HCP_Data_Vis.py:1378: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

/content/HCP_Data_Vis.py:1379: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



FileNotFoundError: [Errno 2] No such file or directory: './1000_Functional_Connectomes/Region Names/Baltimore_5560_region_names_abbrev_file.txt'

#### Creating graphs based on density

In [ ]:
## Create sparser graphs for visualisation - just try them and compare!
# Create sparser graphs based on threshold value
matrix2[matrix2<=0.50] = 0
matrix3[matrix3<=0.75] = 0

st50G = nx.from_numpy_matrix(matrix2)
st25G = nx.from_numpy_matrix(matrix3)

st50G.remove_edges_from(list(nx.selfloop_edges(st50G)))
st25G.remove_edges_from(list(nx.selfloop_edges(st25G)))

# Create sparser graphs based on density - 1%, 2.5%, 5% and 10%, respectively
sd010G = G_den(matrix, 0.010, verbose=False)
sd025G = G_den(matrix, 0.025, verbose=False)
sd050G = G_den(matrix, 0.050, verbose=False)
sd100G = G_den(matrix, 0.100, verbose=False)

#### Let's start with the visualisation of pairwise interactions

In [ ]:
# Create list of values for a property
values = listnet(nx.closeness_centrality, G, distance='distance')

## Visualisation of nodes according to proprieties
# This function will plot the 3D brain with both node size and color according to node_prop
Plot_Brain_Prop(node_prop=values, scale=5) # the second parameter is necessary to scale the node size

In [ ]:
## It is also possible to give a second property, which will change the color of the nodes

# Create a list of a second property
values_2 = listnet(nx.eigenvector_centrality, G, weight='weight')

# Plot 3D brain with node size accoring to values and node colors according to values_2
Plot_Brain_Prop(node_prop=values, scale=5, node_colors=values_2)

In [ ]:
# We have created a degree-specific function for 3D network plotting.
# Node size will always represent the degree/strength, node color can be changed.

# Plotting with both size and colors according to degree (or strength if weight is True)
degree_3D(st50G, scale=0.1, weight=True)

In [ ]:
# Plotting with size according to degree/strength and colors according another property
degree_3D(st25G, scale=0.5, weight=True, node_colors=values, color_prop_name='Closeness Centrality')

In [ ]:
# Visualisation of brain modularity
Plot_Brain_Mod(G, scale=0.5)

In [ ]:
# Visualisation of brain network Participation Rank
dens_value = 0.01 # for plot title
clique_size = 2 # this is a line
alpha = 0.1
plotclique3dk(sd010G, dens_value, clique_size, alpha) # we will use the sd010G to allow a better/clearer visualisation

#### Let's now have a look at high-dimensional interactions (TDA)

In [ ]:
# Visualisation of triangles (or 3-cliques)
dens_value = 0.01 # for plot title
clique_size = 3 # this is a triangle
alpha = 0.1
plotclique3dk(sd010G, dens_value, clique_size, alpha)

In [ ]:
# Visualisation of tetrahedrons (or 4-cliques)
dens_value = 0.025 # for plot title
clique_size = 4 # this is a terahedron
alpha = 0.05
plotclique3dk(sd025G,dens_value, clique_size, alpha)

In [ ]:
# Visualisation of Nodal Curvature
dens_value = 0.025 # for plot title
plotcurv(sd025G, dens_value)

<a id='acknowledgements'></a>

## **4.** Acknowledgements

The 1000_Functional_Connectomes dataset was downloaded from the [The UCLA multimodal connectivity database](http://umcd.humanconnectomeproject.org/).